In [1]:
!uv pip install sagemaker
!uv pip install boto3

Using Python 3.10.15 environment at C:\Users\bored\Music\SagemekerLearning\sagelearner
Audited 1 package in 2.18s
Using Python 3.10.15 environment at C:\Users\bored\Music\SagemekerLearning\sagelearner
Audited 1 package in 12ms


In [3]:
from sagemaker.estimator import Estimator
job_name = 'iris-xgboost-2025-06-08-17-43-29'
xgb_model = Estimator.attach(job_name)
prefix="iris"
bucket = 'sgmkr-learner'

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\bored\AppData\Local\sagemaker\sagemaker\config.yaml

2025-06-08 12:16:32 Starting - Preparing the instances for training
2025-06-08 12:16:32 Downloading - Downloading the training image
2025-06-08 12:16:32 Training - Training image download completed. Training in progress.
2025-06-08 12:16:32 Uploading - Uploading generated training model
2025-06-08 12:16:32 Completed - Training job completed


In [4]:

import os
import boto3
import logging
from datetime import datetime


In [5]:
import sagemaker
from sagemaker.transformer import Transformer

In [6]:
sgmkr_clnt = boto3.client("sagemaker")
sgmkr_rt = boto3.client("runtime.sagemaker")

In [ ]:
import boto3

iam_client = boto3.client('iam')


role_name = "SageMakerExecutionRole"

response = iam_client.get_role(RoleName=role_name)

role_arn = response['Role']['Arn']

print(role_arn)


arn:aws:iam::396913733336:role/SageMakerExecutionRole


In [12]:
model_img="811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest"

In [9]:
model_path = (
    "s3://sgmkr-learner/iris/model/iris-xgboost-2025-06-08-17-43-29/output/model.tar.gz"
)

In [10]:
model_name = "iris-xgboost-" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(model_name)

iris-xgboost-2025-06-09-19-26-51


In [13]:
response = sgmkr_clnt.create_model(
    ModelName=model_name,
    PrimaryContainer={"Image": model_img, "ModelDataUrl": model_path},
    ExecutionRoleArn=role_arn,
)

print(response)

{'ModelArn': 'arn:aws:sagemaker:us-east-1:396913733336:model/iris-xgboost-2025-06-09-19-26-51', 'ResponseMetadata': {'RequestId': 'd3f0dc86-1cd3-4508-90bd-895843bae815', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd3f0dc86-1cd3-4508-90bd-895843bae815', 'content-type': 'application/x-amz-json-1.1', 'content-length': '94', 'date': 'Mon, 09 Jun 2025 13:57:39 GMT'}, 'RetryAttempts': 0}}


In [14]:
batch_ip = 's3://{}/{}/{}'.format(bucket, prefix, 'batch_transform')
batch_op = 's3://{}/{}/{}'.format(bucket, prefix, 'batch_transform')

In [ ]:
model_name = "iris-xgboost-" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S")


In [16]:
ep_config_name = "tmp-ep-config-" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S-%f")
print(ep_config_name)

tmp-ep-config-2025-06-09-19-36-44-083130


In [17]:
response = sgmkr_clnt.create_endpoint_config(
    EndpointConfigName=ep_config_name,
    ProductionVariants=[
        {
            "VariantName": "version-1",
            "ModelName": model_name,
            "InitialInstanceCount": 1,
            "InstanceType": "ml.m4.xlarge",
            # sever_less = ''
        },
    ],
)

print(response)

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:396913733336:endpoint-config/tmp-ep-config-2025-06-09-19-36-44-083130', 'ResponseMetadata': {'RequestId': 'de594c5a-9cfc-44ce-abe5-84d38a2a6beb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'de594c5a-9cfc-44ce-abe5-84d38a2a6beb', 'content-type': 'application/x-amz-json-1.1', 'content-length': '121', 'date': 'Mon, 09 Jun 2025 14:06:56 GMT'}, 'RetryAttempts': 0}}


In [18]:
ep_name = "tmp-ep-" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S-%f")
print(ep_name)

tmp-ep-2025-06-09-19-37-00-114102


In [19]:
response = sgmkr_clnt.create_endpoint(
    EndpointName=ep_name, EndpointConfigName=ep_config_name,
)
print(response)

{'EndpointArn': 'arn:aws:sagemaker:us-east-1:396913733336:endpoint/tmp-ep-2025-06-09-19-37-00-114102', 'ResponseMetadata': {'RequestId': 'd017908b-2dc5-4f5f-8233-69b1ec8ac584', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd017908b-2dc5-4f5f-8233-69b1ec8ac584', 'content-type': 'application/x-amz-json-1.1', 'content-length': '101', 'date': 'Mon, 09 Jun 2025 14:07:20 GMT'}, 'RetryAttempts': 0}}


In [20]:
waiter = sgmkr_clnt.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=ep_name, WaiterConfig={"Delay": 123, "MaxAttempts": 123})
print("Endpoint created")

Endpoint created


In [21]:
sgmkr_runt = boto3.client("runtime.sagemaker")

In [22]:
payload = "7.7, 3.0, 6.1, 2.3"

In [23]:
response = sgmkr_runt.invoke_endpoint(
    EndpointName=ep_name, ContentType="text/csv", Body=payload,
)

prediction = response["Body"].read().decode()
print(prediction)

2.0


In [24]:
print("Deleting sagemaker endpoint")
response = sgmkr_clnt.delete_endpoint(EndpointName = ep_name)  
print("Deleted sagemaker endpoint")
#print(response)   

Deleting sagemaker endpoint
Deleted sagemaker endpoint


In [25]:
print("Deleting sagemaker endpoint configuration")
response = sgmkr_clnt.delete_endpoint_config(EndpointConfigName = ep_config_name)
print("Deleted sagemaker endpoint configuration")

Deleting sagemaker endpoint configuration
Deleted sagemaker endpoint configuration
